In [1]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [38]:
df = pd.read_csv('../monica/monica_weather_2011-2016.csv')
df['date'] = pd.to_datetime(df['de-date'], format='%d.%m.%Y')

In [39]:
df.head()

,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid,date
0,01.01.2011,-6.82,-8.20,-5.45,3.80,2.20,2.78,97.94,2011-01-01
1,02.01.2011,-4.67,-5.85,-3.02,5.48,2.43,0.81,95.94,2011-01-02
2,03.01.2011,-8.42,-13.24,-5.63,2.41,3.33,0.23,96.19,2011-01-03
3,04.01.2011,-11.02,-13.81,-9.68,4.04,2.99,3.20,97.75,2011-01-04
4,05.01.2011,-13.10,-14.60,-10.60,3.73,4.06,0.75,98.50,2011-01-05


In [43]:
def train_prophet(col):
    df_ = df[['date', f'{col}']].copy()
    df_ = df_.rename(columns={"date": "ds", f'{col}': "y"})

    model = Prophet(interval_width=0.95)
    model.fit(df_)
    future = model.make_future_dataframe(periods=365, include_history=False)
    forecast = model.predict(future)
    
    return forecast.yhat_lower.to_numpy(), forecast.yhat.to_numpy(), forecast.yhat_upper.to_numpy()

In [41]:
cols = ['tavg', 'tmin', 'tmax', 'wind', 'globrad', 'precip','relhumid']

In [42]:
test = pd.read_csv('../monica/reanalysis_2017.csv')

In [43]:
test.head()

,de-date,tavg,tmin,tmax,wind,globrad,precip,relhumid
0,01.01.2017,-3.85,-6.03,-1.72,3.96,2.14,0.90,98.50
1,02.01.2017,-1.89,-3.75,-0.85,5.43,1.92,0.57,96.75
2,03.01.2017,-3.72,-9.02,-1.72,5.07,2.04,0.90,95.38
3,04.01.2017,-6.60,-9.16,-5.06,5.27,3.57,3.88,97.56
4,05.01.2017,-4.36,-6.16,-2.20,3.25,3.15,1.17,97.38


In [44]:
for col in cols:
    test[f'{col}_lower'], test[f'{col}_'], test[f'{col}_upper'] = train_prophet(col)

08:24:07 - cmdstanpy - INFO - Chain [1] start processing
08:24:07 - cmdstanpy - INFO - Chain [1] done processing
08:24:09 - cmdstanpy - INFO - Chain [1] start processing
08:24:09 - cmdstanpy - INFO - Chain [1] done processing
08:24:11 - cmdstanpy - INFO - Chain [1] start processing
08:24:11 - cmdstanpy - INFO - Chain [1] done processing
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:12 - cmdstanpy - INFO - Chain [1] done processing
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:13 - cmdstanpy - INFO - Chain [1] done processing
08:24:13 - cmdstanpy - INFO - Chain [1] start processing
08:24:13 - cmdstanpy - INFO - Chain [1] done processing
08:24:14 - cmdstanpy - INFO - Chain [1] start processing
08:24:15 - cmdstanpy - INFO - Chain [1] done processing


In [45]:
# Lower
lower = test[[
    'de-date', 'tavg_lower', 'tmin_lower', 'tmax_lower', 'wind_lower', 'globrad_lower', 'precip_lower', 'relhumid_lower'
]]
lower = lower.rename(columns={
    "tavg_lower":"tavg", "tmin_lower":"tmin", "tmax_lower":"tmax", "wind_lower":"wind", "globrad_lower":"globrad",
    "precip_lower":"precip", "relhumid_lower":"relhumid"
})
lower.to_csv(
    '../monica/prophet_2017_lower.csv', index=False
)

# Avg
avg = test[[
    'de-date', 'tavg_', 'tmin_', 'tmax_', 'wind_', 'globrad_', 'precip_', 'relhumid_'
]]
avg = avg.rename(columns={
    "tavg_":"tavg", "tmin_":"tmin", "tmax_":"tmax", "wind_":"wind", "globrad_":"globrad",
    "precip_":"precip", "relhumid_":"relhumid"
})
avg.to_csv(
    '../monica/prophet_2017.csv', index=False
)

# Upper
upper = test[[
    'de-date', 'tavg_upper', 'tmin_upper', 'tmax_upper', 'wind_upper', 'globrad_upper', 'precip_upper', 'relhumid_upper'
]]
upper = upper.rename(columns={
    "tavg_upper":"tavg", "tmin_upper":"tmin", "tmax_upper":"tmax", "wind_upper":"wind", "globrad_upper":"globrad",
    "precip_upper":"precip", "relhumid_upper":"relhumid"
})
upper.to_csv(
    '../monica/prophet_2017_upper.csv', index=False
)

In [44]:
df = pd.read_csv('../wofost/reanalysis_2011_2016.csv')
df["date"] = pd.to_datetime(df["DAY"], format='%Y%m%d')

In [45]:
cols = ['IRRAD', 'TMIN', 'TMAX', 'VAP', 'WIND', 'RAIN']

In [46]:
test = pd.read_csv('../wofost/reanalysis_2017.csv')

In [47]:
for col in cols:
    test[f'{col}_lower'], test[f'{col}_'], test[f'{col}_upper'] = train_prophet(col)

07:57:08 - cmdstanpy - INFO - Chain [1] start processing
07:57:11 - cmdstanpy - INFO - Chain [1] done processing
07:57:13 - cmdstanpy - INFO - Chain [1] start processing
07:57:16 - cmdstanpy - INFO - Chain [1] done processing
07:57:19 - cmdstanpy - INFO - Chain [1] start processing
07:57:23 - cmdstanpy - INFO - Chain [1] done processing
07:57:25 - cmdstanpy - INFO - Chain [1] start processing
07:57:29 - cmdstanpy - INFO - Chain [1] done processing
07:57:32 - cmdstanpy - INFO - Chain [1] start processing
07:57:35 - cmdstanpy - INFO - Chain [1] done processing
07:57:37 - cmdstanpy - INFO - Chain [1] start processing
07:57:39 - cmdstanpy - INFO - Chain [1] done processing


In [48]:
test.head()

,DAY,IRRAD,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH,IRRAD_lower,IRRAD_,...,TMAX_upper,VAP_lower,VAP_,VAP_upper,WIND_lower,WIND_,WIND_upper,RAIN_lower,RAIN_,RAIN_upper
0,20170101,2140.0,-6.31,-1.60,0.443766,3.77,0.79,NaN,-5433.149912,2037.069425,...,2.918251,0.069425,0.485782,0.915126,1.372345,3.788480,6.337239,-4.935851,2.760000,10.130844
1,20170102,1920.0,-4.31,-0.91,0.491824,5.28,0.35,NaN,-5348.077663,1973.380884,...,2.973583,0.041166,0.478912,0.885667,1.415359,3.816114,6.097797,-4.341632,2.665275,9.669179
2,20170103,2040.0,-6.40,-1.92,0.430543,5.09,0.74,NaN,-5652.475335,1884.803965,...,2.178658,0.023117,0.461965,0.870814,1.221819,3.675831,6.032918,-4.013099,2.799190,10.207034
3,20170104,3570.0,-7.85,-4.60,0.360791,5.46,2.64,NaN,-5374.276792,2273.528255,...,1.839778,-0.040506,0.444488,0.869254,1.225266,3.549072,5.842457,-4.594579,2.430930,9.855411
4,20170105,3150.0,-5.21,-1.74,0.434473,3.29,1.52,NaN,-5850.617740,2116.660934,...,1.989322,-0.002805,0.444387,0.897652,1.217971,3.538513,6.009417,-3.957641,2.897817,9.931628


In [49]:
# Lower
lower = test[[
    'DAY', 'IRRAD_lower', 'TMIN_lower', 'TMAX_lower', 'VAP_lower', 'WIND_lower', 'RAIN_lower'
]]
lower = lower.rename(columns={
    "IRRAD_lower":"IRRAD", "TMIN_lower":"TMIN", "TMAX_lower":"TMAX", "VAP_lower":"VAP", "WIND_lower":"WIND",
    "RAIN_lower":"RAIN"
})
lower.to_csv(
    '../wofost/prophet_2017_lower.csv', index=False
)

# Avg
avg = test[[
    'DAY', 'IRRAD_', 'TMIN_', 'TMAX_', 'VAP_', 'WIND_', 'RAIN_'
]]
avg = avg.rename(columns={
    "IRRAD_":"IRRAD", "TMIN_":"TMIN", "TMAX_":"TMAX", "VAP_":"VAP", "WIND_":"WIND",
    "RAIN_":"RAIN"
})
avg.to_csv(
    '../wofost/prophet_2017.csv', index=False
)

# Upper
upper = test[[
    'DAY', 'IRRAD_upper', 'TMIN_upper', 'TMAX_upper', 'VAP_upper', 'WIND_upper', 'RAIN_upper'
]]
upper = upper.rename(columns={
    "IRRAD_upper":"IRRAD", "TMIN_upper":"TMIN", "TMAX_upper":"TMAX", "VAP_upper":"VAP", "WIND_upper":"WIND",
    "RAIN_upper":"RAIN"
})
upper.to_csv(
    '../wofost/prophet_2017_upper.csv', index=False
)